In [ ]:
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torchvision import models
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import matplotlib.pyplot as plt
train_on_gpu = True

In [ ]:
# !pip install syft

In [ ]:
# import syft as sy
# import torch as th
# hook = sy.TorchHook(th)
# bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
# alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
# secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [ ]:
train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

In [ ]:
class APTOSDataset(Dataset):
  def __init__(self,df,datatype,transform,y):
    self.df = df
    self.transform = transform
    self.datatype = datatype
    self.image_files_list = [f'../input/aptos2019-blindness-detection/{self.datatype}_images/{i}.png' for i in df['id_code'].values]
    self.labels = y
    
  def __len__(self):
    return len(self.image_files_list) 
   
  def __getitem__(self,idx):
    img_name = self.image_files_list[idx]
    img = cv2.imread(img_name)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    image = self.transform(img)
    label = self.labels[idx]
    return image,label

In [ ]:
train_transforms = transforms.Compose([transforms.ToPILImage(),
                                       transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


valid_transforms = transforms.Compose([transforms.ToPILImage(),
                                       transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [ ]:
trainset, validation_set = train_test_split(train, stratify=train.diagnosis, test_size=0.20)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
def prepare_labels(y):
    # From here: https://www.kaggle.com/pestipeti/keras-cnn-starter
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

    y = onehot_encoded
    return y, label_encoder
y_train, le = prepare_labels(trainset['diagnosis'])
y_valid, le = prepare_labels(validation_set['diagnosis'])

In [ ]:
train_data = APTOSDataset(df=trainset, datatype="train", transform=train_transforms,y=y_train)
valid_data = APTOSDataset(df=validation_set, datatype="train", transform=valid_transforms,y=y_train)

In [ ]:
import torch
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64)

In [ ]:
model = models.resnet50(pretrained=True)
model

In [ ]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.fc = nn.Linear(2048, 5)
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.99)

In [ ]:
n_epochs = 20
for epoch in range(1, n_epochs+1):

    train_loss = []
    train_auc = []

    for batch_i, (data, target) in enumerate(train_loader):

        data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.float())
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    
    model.eval()
    val_loss = []
    val_auc = []
    for batch_i, (data, target) in enumerate(valid_loader):
        data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target.float())
        val_loss.append(loss.item()) 
    model.train()
    # print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}, train auc: {np.mean(train_auc):.4f}, valid auc: {np.mean(val_auc):.4f}')
    print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}.')